In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
import h5py

from analysis.lib.m2 import m2

In [ ]:
def get_minfreq_from_folder(folder):
    try: 
        minfreq = int(os.path.split(folder)[-1][-4:])
    except:
        try:
            minfreq = int(os.path.split(folder)[-1][-3:])
        except:
            try:
                minfreq = int(os.path.split(folder)[-1][-2:])
            except: 
                try:
                    minfreq = int(os.path.split(folder)[-1][-1:])
                except: 
                    return 1000
    return minfreq 


def plot_data(folder,wavemeter_after_stab, wavemeter,jpe,readout,readout_PSB,int_times, show_plots=True, name =''):
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(wavemeter_after_stab,jpe,'or')
    ax.set_ylabel('jpe voltage (V)',color='r',fontsize=14)
    ax2 = ax.twinx()
    ax2.plot(wavemeter_after_stab,wavemeter_after_stab,'o')
    ax2.set_ylabel('laser freq (GHZ)',color = 'b',fontsize=14)
    ax.set_xlabel('laser freq (GHz)',fontsize=14)
    plt.savefig(os.path.join(folder,'jpe_V'+name+'.png'))
    if show_plots:
        plt.show()
    plt.close()
    

    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(wavemeter_after_stab,readout/int_times,'o')
    ax.set_ylabel('readout rate in ZPL (Hz)',color = 'b',fontsize=14)
    ax.set_xlabel('laser freq (GHz)',fontsize=14)
    plt.savefig(os.path.join(folder,'readout_rate'+name+'.svg'))
    if show_plots:
        plt.show()
    plt.close()

    
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(wavemeter_after_stab,readout_PSB/int_times,'o')
    ax.set_ylabel('readout rate in PSB (Hz)',color = 'b',fontsize=14)
    ax.set_xlabel('laser freq (GHz)',fontsize=14)
    plt.savefig(os.path.join(folder,'readout_rate_PSB'+name+'.svg'))
    if show_plots:
        plt.show()
    plt.close()
    
def plot_stab_success(folder, stab_success,stab_rounds,nr_points,show_plots=True, name =''):
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(np.arange(nr_points),stab_success,'or')
    ax.set_ylabel('fail',color='r',fontsize=14)
    ax.set_xlabel('step nr',fontsize=14)
    plt.savefig(os.path.join(folder,'stab_successes'+name+'.png'))
    if show_plots:
        plt.show()
    plt.close()
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(np.arange(nr_points),stab_rounds,'or')
    ax.set_ylabel('rounds',color='r',fontsize=14)
    ax.set_xlabel('step nr',fontsize=14)
    plt.savefig(os.path.join(folder,'stab_rounds'+name+'.png'))
    if show_plots:
        plt.show()
    plt.close()

def bin_data(freq_min,freq_max,wavemeter,readout_PSB,binsize=0.05):
    nr_bins = int((freq_max-freq_min)/binsize)+1
    bins = np.linspace(freq_min,freq_max,nr_bins)

    _h_points,_b_points = np.histogram(wavemeter, weights=np.ones(len(readout_PSB)), bins=bins)
#     _h_points[np.where(_h_points==0)] = 1
    _h,_b = np.histogram(wavemeter, weights=readout_PSB, bins=bins)

    return _b,_h,_h_points
    



In [ ]:
def get_folder_long_scan(older_than=None):
    folders = []
    timestamp, folder1 = tb.latest_data('AdwinMeasurement_test_80', return_timestamp=True, older_than=older_than)
    folders.append(folder1)
    
    minfreq  = get_minfreq_from_folder(folder1)
    print minfreq
    i=0
    try_next=True
    while try_next == True:
        timestamp, folder_next = tb.latest_data('AdwinMeasurement', older_than = timestamp,return_timestamp=True)
        minfreq_next  = get_minfreq_from_folder(folder_next)

        if minfreq_next < minfreq:
            folders.append(folder_next)
        else:
            try_next = False
        i+=1
        minfreq=minfreq_next
        if i > 20:
            break
    return folders 

def get_folder_vary_powers(nr_folders=1,older_than=None):
    folders = []
    timestamp, folder1 = tb.latest_data('AdwinMeasurement', return_timestamp=True,older_than=older_than)
    folders.append(folder1)
    print folder1
    i=1
    try_next=True
    while try_next == True:
        if i >= nr_folders:
            try_next=False
            break #doub;e , but alright
        timestamp, folder_next = tb.latest_data('AdwinMeasurement', older_than = timestamp,return_timestamp=True)
        print folder_next
        i+=1
        folders.append(folder_next)
        if i > 20:
            break
    return folders

In [ ]:
# folders = get_folder_vary_powers(nr_folders=1, older_than=None)#,older_than='20180110131705')
folders = get_folder_long_scan()
#print folders
freq_min=-100
freq_max=100
binsize=0.05


fig,ax=plt.subplots()

for i,folder in enumerate(folders):
# folder= tb.latest_data('AdwinMeasurement_test_-50')
    filepath = tb.measurement_filename(folder)

    name = 'Giuseppe'
    try: 
        a = m2.M2Analysis(folder)
    except:
        a.finish()
        next
    print filepath

    jpe = a.g['step_and_stable']['jpe_voltage'].value
    wavemeter_after_stab = a.g['step_and_stable']['wavemeter_readout'].value
    #wavemeter = a.g['step_and_stable']['wavemeter_readout_per_round'].value
    readout = a.g['step_and_stable']['readout_counts'].value
    readout_PSB = a.g['step_and_stable']['PSB_readout_counts'].value
    stab_success = a.g['step_and_stable']['stab_success'].value
    stab_rounds = a.g['step_and_stable']['stab_rounds'].value
    #rounds_per_step = a.g['step_and_stable']['rounds_per_step'].value

#     print wavemeter
#     print wavemeter_after_stab
#     print readout
#     print readout_PSB
#     print rounds_per_step
    
    int_time_start = a.g.attrs['integration_time']
    int_time_stop = a.g.attrs['integration_time_stop']
    nr_points = a.g.attrs['nr_steps']
    int_times = np.linspace(int_time_start,int_time_stop,len(readout))#dirty hack: should have each time, times nr rounds per step#
    
    if i == 0:
        jpe_all=jpe
        readout_all=readout
        readout_PSB_all=readout_PSB
        stab_success_all=stab_success
        stab_rounds_all=stab_rounds
#         wavemeter_all = wavemeter
        wavemeter_after_stab_all = wavemeter_after_stab
        int_times_all = int_times
        nr_points_all = nr_points
    else:
        jpe_all = np.append(jpe_all,jpe)
        readout_all = np.append(readout_all, readout)
        readout_PSB_all = np.append(readout_PSB_all, readout_PSB)
        stab_success_all = np.append(stab_success_all,stab_success)
        stab_rounds_all = np.append(stab_rounds_all,stab_rounds)
#         wavemeter_all = np.append(wavemeter_all,wavemeter)
        wavemeter_after_stab_all = np.append(wavemeter_after_stab_all,wavemeter_after_stab)
        int_times_all = np.append(int_times_all,int_times)
        nr_points_all += nr_points

#     _b,_h,_h_points = bin_data(freq_min,freq_max,wavemeter,readout_PSB,binsize=binsize)
    
#     fig,ax=plt.subplots()
#     ax.plot(_b[:-1]+np.diff(_b), np.divide(_h,_h_points)/int_times[0],'blue')
#     ax.set_xlabel('frequency (GHz)')
#     ax.set_ylabel('readout counts (Hz)')
#     ax.set_title(folder+'b_%.2f'%(binsize))
#     plt.savefig(os.path.join(folder,'histogramcounts_b_%.2f'%(binsize)+'.png'))
# #     ax.set_xlim(-30,-25)
#     plt.show()
#     plt.close()#     plt.plot(_b[1:], _h/_h_points,'blue')
    
#     if i==0:
#         _h_points_all = _h_points
#         _h_all = _h
#     else:
#         _h_points_all+=_h_points
#         _h_all += _h
#     print i
#     if (i+1)%4==0:
        
        
#         fig,ax=plt.subplots()
#         ax.plot(_b[:-1]+np.diff(_b), np.divide(_h_all,_h_points),'blue')
#         ax.set_xlabel('frequency (GHz)')
#         ax.set_ylabel('readout counts (total)')
#         plt.show()
#         plt.close()
 
#     L=13*2.41+3.6 #in um
#     nu = 470.4e3+wavemeter[0]
#     conversion = 65e-6 #um/mV

#     plot_data(folder,wavemeter_after_stab,wavemeter,jpe,readout,readout_PSB,int_times,show_plots=True)

#     plot_stab_success(folder, stab_success,stab_rounds,nr_points,show_plots=False)
    
#     a.finish()
 
    
# fig,ax=plt.subplots()
# ax.plot(_b[:-1]+np.diff(_b), np.divide(_h_all,_h_points_all)/int_times[0],'blue')
# ax.set_xlabel('frequency (GHz)')
# ax.set_ylabel('readout counts (Hz)')
# ax.set_title(folder+'all_b_%.2f'%(binsize))
# plt.savefig(os.path.join(folder,'histogramcounts_all_b_%.2f'%(binsize)+'.png'))
# plt.show()
# plt.close()   
wavemeter_all= wavemeter_after_stab_all
plot_data(folder, wavemeter_after_stab_all, wavemeter_all,jpe_all,readout_all,readout_PSB_all,int_times_all,show_plots=True,name='all')
plot_stab_success(folder, stab_success_all,stab_rounds_all,nr_points_all,show_plots=True,name='all')


In [ ]:
print _h_all
print _h_points_all
fig,ax=plt.subplots()
ax.plot(_b[:-1]+np.diff(_b), _h_all/_h_points_all/int_times[0],'blue')
ax.set_xlabel('frequency (GHz)')
ax.set_ylabel('readout counts (Hz)')
# ax.set_xlim(-36,-34)
plt.show()
plt.close()




In [ ]:
plot_stab_success
for folder in folders:
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(np.arange(nr_points),stab_success,'or')
    ax.set_ylabel('fail',color='r',fontsize=14)
    ax.set_xlabel('step nr',fontsize=14)
    plt.savefig(os.path.join(folder,'readout_rate.png'))
    plt.show()
    fig,ax=plt.subplots()
    ax.set_title(folder)
    ax.plot(np.arange(nr_points),stab_rounds,'or')
    ax.set_ylabel('rounds',color='r',fontsize=14)
    ax.set_xlabel('step nr',fontsize=14)
    plt.savefig(os.path.join(folder,'readout_rate.png'))
    plt.show()




In [ ]:
########binning data
nr_bins = 10
wavemeterCut = wavemeter[len(wavemeter)%nr_bins:]
readoutCut= readout[len(wavemeter)%nr_bins:]
jpeCut= jpe[len(wavemeter)%nr_bins:]
wavemeterBinned = np.mean(wavemeterCut.reshape(-1, nr_bins), axis=1)
readoutBinned =np.mean(readoutCut.reshape(-1, nr_bins), axis=1)
jpeBinned =np.mean(jpeCut.reshape(-1, nr_bins), axis=1)


In [ ]:
fig,ax=plt.subplots()
ax.set_title(folder)
ax.plot(wavemeterBinned,readoutBinned/int_times[0],'-r')
ax.set_ylabel('readout rate (Hz)',color='r',fontsize=14)
ax.set_xlabel('laser freq (GHz)',fontsize=14)
ax2 = ax.twinx()
ax2.plot(wavemeterBinned,jpeBinned,'-')
ax2.set_ylabel('jpe V',color = 'b',fontsize=14)
plt.savefig(os.path.join(folder,'readout_rate_binned.png'))
plt.show()

In [ ]:

folder= tb.latest_data('191716_AdwinMeasurement_test1')
print folder
a = m2.M2Analysis(folder)
jpe = a.f['test1']['step_and_stable']['jpe_voltage'].value
wavemeter = a.f['test1']['step_and_stable']['wavemeter_readout'].value

L=13*2.41+3.6 #in um
nu = 470.4e3+wavemeter[0]
conversion = 65e-6 #um/mV

dnu = (jpe-jpe[0])*1000*conversion/L*nu#in GHz



fig,ax=plt.subplots()
ax.plot(dnu,'or')
ax.set_ylabel('frequency change (V)',color='r',fontsize=14)
ax2 = ax.twinx()
ax2.plot(wavemeter,'o')
ax2.set_ylabel('laser freq (GHZ)',color = 'b',fontsize=14)
ax2.set_xlabel('step nr',fontsize=14)
plt.show()
print i
#     times = f['fidelity/time'].value
#     fids0 = f['fidelity/ms0'].value
#     fids1 = f['fidelity/ms' + str(el_state[-2:])].value